In [1]:
import os
import site
import sqlite3
import sys

import logzero
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yaml
from logzero import logger
from tqdm import tqdm
from tqdm.notebook import tqdm
from yaml import dump, load, safe_load

In [2]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash_table import DataTable
from jupyter_dash import JupyterDash

In [3]:
sys.path.append("../../sql")
import queries

sys.path.append("../source")
import dbrd_eda
import dbrd_tools
import plot_tools
import ts_tools

In [4]:
log_path = "logs/"
log_file = "dashboard_eda.log"

logzero.logfile(log_path + log_file, maxBytes=1e6, backupCount=5, disableStderrLogger=True)
logger.info(f"{log_path}, {log_file}\n")

In [5]:
configs = None
try:
    with open("../configs/config.yml", "r") as config_in:
        configs = load(config_in, Loader=yaml.SafeLoader)
        logger.info(f"{configs}\n")
except:
    logger.error(f"config file open failure.")
    exit(1)

cfg_vars = configs["url_variables"]
logger.info(f"variables: {cfg_vars}\n")

years = configs["request_years"]
logger.info(f"years: {years}\n")

db_path = configs["file_paths"]["db_path"]

city = configs["location_info"]["city"]
state = configs["location_info"]["state"]
db_file = city + "_" + state + ".db"

db_table1 = configs["table_names"]["db_table1"]
db_table2 = configs["table_names"]["db_table2"]

logger.info(f"{db_path}, {db_file}")

data_units = configs["data_units"]
meteoro_fields = configs["meteorological_fields"]

nrows = configs["num_rows"][0]
logger.info(f"number of rows: {nrows}\n")

In [6]:
db_files = ts_tools.get_db_files(db_path)
logger.info(f"DB Path: {db_path}\n{db_files}\n")

In [7]:
def get_db_connection(db_filename=db_file):
    conn = sqlite3.connect(db_path + db_filename)
    logger.info(f"Connection made: {conn}")
    return conn

In [8]:
def get_db_zipcodes(conn):
    cursor = conn.cursor()
    cursor.execute(queries.select_distinct_zips)
    zipcodes = cursor.fetchall()
    zipcodes = [z[0] for z in zipcodes]
    logger.info(f"Distinct zip codes: {zipcodes}")
    return zipcodes

In [9]:
zipcode_index = 0

conn = get_db_connection(db_filename=db_file)
zipcodes = get_db_zipcodes(conn)

df = ts_tools.get_irr_data(conn, zipcodes[zipcode_index])

df_rsm = df.resample("M").mean()
# df_rsm.head(5)

In [10]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE])

server = app.server

In [11]:
app.layout = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(
                    dcc.Dropdown(
                        id="dd-db-selection",
                        options=[{"label": db, "value": db} for db in db_files],
                        value=db_files[0],
                        placeholder="Select a database",
                    ),
                    width={"size": 2, "offset": 1},
                ),
                dbc.Col(
                    dcc.Dropdown(
                        id="dd-zipcode-selection",
                        placeholder="Select a Zip Code",
                    ),
                    width={"size": 2, "offset": 1},
                ),
                dbc.Col(
                    dcc.Dropdown(
                        id="dd-viz-selection",
                        placeholder="Select a Viz Page Code",
                    ),
                    width={"size": 2, "offset": 1},
                ),

            ],
            no_gutters=False,
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label(
                            "Data View",
                            style={"display": "inline-block", "textAlign": "center"},
                        ),
                        dcc.Graph(
                            id="graph-data-view",
                        ),
                    ],
                    width={"size": 5, "offset": 1},
                ),
                dbc.Col(
                    [
                        html.Label(
                            "Distribution View",
                            style={"display": "inline-block", "textAlign": "center"},
                        ),
                        dcc.Graph(
                            id="graph-dist-view",
                        ),
                    ],
                    width={"size": 5},
                ),
            ],
            no_gutters=True,
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label(
                            "Meteorological View",
                            style={"display": "inline-block", "textAlign": "center"},
                        ),
                        dcc.Graph(
                            id="graph-meteoro-view",
                        ),
                    ],
                    width={"size": 5, "offset": 1},
                ),
                dbc.Col(
                    [
                        html.Label(
                            "Desciptive Statistics",
                            style={
                                "display": "inline-block",
                                "textAlign": "center",
                            },
                        ),
                        DataTable(
                            id="table-desc-stats",
                            style_cell={
                                "padding": "5px",
                                "backgroundColor": "black",
                                "forgroundColor": "white",
                                "fontWeight": "bold",
                            },
                            style_header={
                                "padding": "5px",
                                "backgroundColor": "black",
                                "forgroundColor": "white",
                                "fontWeight": "bold",
                            },
                        ),
                    ],
                    width={"size": 5},
                ),
            ],
        ),
    ],
)

# end of layout
#
#
# begin callbacks


@app.callback(
    Output("dd-zipcode-selection", "options"),
    [
        Input("dd-db-selection", "value"),
    ],
)
def get_zipcodes(file_name):
    logger.info(f"get_zipcodes callback: {file_name}")

    # cntx = dash.callback_context
    # context = cntx.triggered[0]["prop_id"].split(".")[0]
    # print(f"get_zipcodes Context = {context}")

    conn = get_db_connection(db_filename=file_name)
    zipcodes = get_db_zipcodes(conn)
    conn.close()

    # return the list object to properly populate the dropdown!
    return [{"label": zipcode, "value": zipcode} for zipcode in zipcodes]


@app.callback(
    Output("dd-zipcode-selection", "value"),
    [
        Input("dd-zipcode-selection", "options"),
    ],
)
def set_zipcode_value(options):
    return options[0]["value"]


@app.callback(
    [
        Output("graph-data-view", "figure"),
        Output("graph-dist-view", "figure"),
        Output("graph-meteoro-view", "figure"),
        Output("table-desc-stats", "data"),
        Output("table-desc-stats", "columns"),
    ],
    [
        Input("dd-db-selection", "value"),
        Input("dd-zipcode-selection", "value"),
    ],
)
def graph_output(db_filename, zipcode):

    cntx = dash.callback_context
    context = cntx.triggered[0]["prop_id"].split(".")[0]
    # print(f"graph_output #1 Context = {context}\n")

    if context == "dd-db-selection":
        conn = ts_tools.get_db_connection(db_path, db_filename)
        zipcodes = ts_tools.get_db_zipcodes(conn)
        zipcode = zipcodes[0]
        df = ts_tools.get_irr_data(conn, zipcode)
        # print(f"Made if: {db_filename}, {zipcode}")

    elif context == "dd-zipcode-selection":
        # print(f"Made elif: {db_filename}, {zipcode}")
        conn = ts_tools.get_db_connection(db_path, db_filename)
        df = ts_tools.get_irr_data(conn, zipcode)

    else:
        db_filename = db_files[0]
        conn = ts_tools.get_db_connection(db_path, db_filename)

        zipcodes = ts_tools.get_db_zipcodes(conn)
        zipcode = zipcodes[0]

        df = ts_tools.get_irr_data(conn, zipcode)
        # print(f"Made else: {db_filename}, {zipcode}")

    # print(f"graph_output #2 Context = {context}, File Name {db_filename}, Zipcode {zipcode}\n")

    df_rsm = df.resample("M").mean()
    
    df_desc = df_rsm.describe().transpose().round(decimals=2).reset_index(drop=False)
    desc_columns = [{"id": col, "name": col} for col in df_desc.columns]

    title1 = "Irradiance Data"
    fig1 = plot_tools.plot_data(
        df_rsm,
        title=title1,
        zipcode=zipcode,
        units=data_units,
    )

    title2 = "Data Distributions"
    fig2 = plot_tools.plot_histograms(
        df_rsm,
        title=title2,
        zipcode=zipcode,
    )

    title3 = "Meteorological Conditions"
    fig3 = plot_tools.plot_multi_line(
        df_rsm,
        title=title3,
        columns=meteoro_fields,
    )

    return fig1, fig2, fig3, df_desc.to_dict("records"), desc_columns

In [12]:
if __name__ == "__main__":
    app.run_server(mode="external", host="192.168.64.164", port="8080", use_reloader=False, debug=True)

Dash app running on http://192.168.64.164:8080/
